# Example showing how mix python and madagascar processing #

This notebook:
    * creates spike.rsf data using sfspike abnd sfbandlimit
    * opens the spike.rsf file as input
    * open a new clip.rsf file as output
    * one trace at a time, read, clip, write
    * repeats this processing using the File class
    * shows how to create and sconstruct file and use it
    
The notebook can be run in Python 3 (using git updates from m8r.py and doc.py on July 27, 2018)
and Madagascar built with Python 2.  m8r.py has been tested for both Python 2 and Python 3.

# First initialize.  
Import os, numpy, matplotlib, matplotlib.pyplot, and m8r

In [ ]:
import os
import numpy as np
import matplotlib
# use the nbagg backend to render plot.  It provides basic zoom and pan capability.
matplotlib.use('nbagg') # previously I used this magic command: %matplotlib inline
# I think import matplotlib.pyplot must be after matplotlib.use('nbagg') 
import matplotlib.pyplot as plt

#m8r library provides read and write of madagascar data
import m8r 

In [ ]:
os.system('sfspike n1=1000 n2=100 n3=10 nsp=1 k1=500 | sfbandpass fhi=20 phase=y > spike.rsf') 

In [ ]:
inp  = m8r.Input('spike.rsf')
output = m8r.Output('clip.rsf')

n1 = inp.int("n1")
n2 = inp.size(1)

clip = 0.05

for i2 in xrange(n2): # loop over traces
    # read one trace using the shape parameter
    # data type comes from input file.  This data is single precision float
    trace=inp.read(shape=(n1))
    trace=np.clip(trace,-clip,clip)
    output.write(trace)
output.close()

# Process spike.rsf using M8r.File class
inp is now a m8r.file object

trace=inp[:,i2,i3] creates an ndarray of the trace at location i2,l3

In [ ]:
inp = m8r.File('spike.rsf')
output = m8r.Output('clip.rsf')

clip = 0.05

# python     indicies are slowest to fastest.  
# Madagascar indicies are fastest to slowest
n3,n2,n1 = inp.shape()

output.put("n1",n1)
output.put("n2",n2)
output.put("n3",n3)

for i3 in xrange(n3): # loop over slices
    for i2 in xrange(n2): # loop over traces
        trace=inp[i3,i2,:]
        trace = np.clip(trace,-clip,clip)
        output.write(trace)

In [ ]:
# inp is a File object:
type(inp)

In [ ]:
# the file object can be sliced to create one dimention numpy ndarray
trace=inp[0,0,:]
print ('inp.shape(0)='+repr(inp.shape()))

In [ ]:
# file objects have the method min, max, mean
(inp.min(),inp.max(),inp.mean())

In [ ]:
# plot spike.rsf using sfwiggle
os.system('sfwiggle <spike.rsf | sfpen&')
# plot clip.rsf using sfwiggle
os.system('sfwiggle <clip.rsf | sfpen&')

In [ ]:
# Any terminal command can be run in jupyter by prefixing it with the ! character.
!sfwiggle <spike.rsf | sfpen

In [ ]:
# appending [:] to the end of a File object converts to a numpy array.
print('type(inp[:]='+repr(type(inp[:])) +'   inp[:].shape='+repr(inp[:].shape))

In [ ]:
# you can convert a file to numpy array and use numpy function np.clip
clipped = np.clip(inp[:],-clip,clip)
# and convert a numpy array to a File object. Filename is temporary.  
clipfile = m8r.File(clipped)

In [ ]:
clipfile.disfil


In [ ]:
#madagascar programs can be called using m8r.py filters.  Filters can be chained:
clipfile = m8r.bandpass(fhi=2).clip(clip=0.05)[inp]

In [ ]:
#smooth division of clipfile by inp:
added = m8r.divn(rect1=10,den=inp)[clipfile]

In [ ]:
#select a line from a 3D volume and plot:
sliced = m8r.window(n3=1)[clipfile]
sliced.grey(title='Slice')

In [ ]:
# to run scons first create the SConstruct file:

In [ ]:
%%file test.scons

Flow('spike2','spike','math output="exp(input)"')

Result('spike2','window n3=1 f3=5 | grey title="Exp" pclip=100 allpos=y')

In [ ]:
#then run using m8r.view
m8r.view('spike2')

In [ ]:
#you can creae a .c file and compile it to make a program

In [ ]:
%%file clip.c

/* Clip the data. */

#include <rsf.h>

int main(int argc, char* argv[])
{
    int n1, n2, i1, i2;
    float clip, *trace;
    sf_file in, out; /* Input and output files */

    /* Initialize RSF */
    sf_init(argc,argv);
    /* standard input */
    in  = sf_input("in");
    /* standard output */
    out = sf_output("out");

    /* check that the input is float */
    if (SF_FLOAT != sf_gettype(in)) 
	sf_error("Need float input");

    /* n1 is the fastest dimension (trace length) */
    if (!sf_histint(in,"n1",&n1)) 
	sf_error("No n1= in input");
    /* leftsize gets n2*n3*n4*... (the number of traces) */
    n2 = sf_leftsize(in,1);

    /* parameter from the command line (i.e. clip=1.5 ) */
    if (!sf_getfloat("clip",&clip)) sf_error("Need clip=");

    /* allocate floating point array */
    trace = sf_floatalloc (n1);

    /* loop over traces */
    for (i2=0; i2 < n2; i2++) {

	/* read a trace */
	sf_floatread(trace,n1,in);

	/* loop over samples */
	for (i1=0; i1 < n1; i1++) {
	    if      (trace[i1] >  clip) trace[i1]= clip;
	    else if (trace[i1] < -clip) trace[i1]=-clip;
	}

	/* write a trace */
	sf_floatwrite(trace,n1,out);
    }


    exit(0);
}

In [ ]:
m8r.Fetch('shots.hh','shots')

In [ ]:
m8r.view('spike2')

In [ ]:
inp.close()
inp=m8r.Input("spike.rsf")
alltraces=inp.read()


In [ ]:
# remove '#' on next line to dump part of the data
#alltraces

In [ ]:
traces = m8r.put(d1=0.004,d2=0.1)[m8r.File(alltraces)]

In [ ]:
#select a 2D slice from the 3D array alltraces, plot it with imshow seismic data - fastest axis down
plt.imshow(alltraces[0,:,:].T,aspect='auto')
plt.show()

#experiment with zoom plot 

In [ ]:
patch = m8r.patch(w=[200,50,5],p=[4,2,2])[inp]
inp2 = m8r.patch(inv=True,weight=True)[patch]

In [ ]:
inp2.shape()

In [ ]:
allclip = np.clip(alltraces,-clip,clip)
#select a 2D slice from the 3D array alltraces, plot it with imshow seismic data - fastest axis down
plt.imshow(allclip[0,:,:].T,aspect='auto')
plt.show()

#experiment with zoom plot 